In [2]:
import torch
from torch import nn
from torch.optim import SGD
from torch import Tensor
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
def train_loop(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss, optimizer: SGD) -> None:
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred: Tensor = model(X)
        loss: Tensor = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred: Tensor = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301590  [    0/60000]
loss: 2.291309  [ 6400/60000]
loss: 2.268839  [12800/60000]
loss: 2.265604  [19200/60000]
loss: 2.253871  [25600/60000]
loss: 2.215109  [32000/60000]
loss: 2.236308  [38400/60000]
loss: 2.193740  [44800/60000]
loss: 2.182660  [51200/60000]
loss: 2.153197  [57600/60000]
Test Error: 
 Accuracy: 42.2%, Avg loss: 2.149939 

Epoch 2
-------------------------------
loss: 2.156206  [    0/60000]
loss: 2.152731  [ 6400/60000]
loss: 2.088343  [12800/60000]
loss: 2.110292  [19200/60000]
loss: 2.061074  [25600/60000]
loss: 1.988632  [32000/60000]
loss: 2.028625  [38400/60000]
loss: 1.938987  [44800/60000]
loss: 1.933419  [51200/60000]
loss: 1.869789  [57600/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.871122 

Epoch 3
-------------------------------
loss: 1.897760  [    0/60000]
loss: 1.876000  [ 6400/60000]
loss: 1.754604  [12800/60000]
loss: 1.803014  [19200/60000]
loss: 1.691385  [25600/60000]
loss: 1.639505  [32000/600